In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import random
import time

import warnings
warnings.filterwarnings("ignore")

np.random.seed(0)
random.seed(0)

In [ ]:
# Set the CUDA_VISIBLE_DEVICES environment variable to the index of the GPU you want to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Use GPU 0 (change the index as needed)

# Now, TensorFlow will only see the GPU you specified
gpus = tf.config.list_physical_devices('GPU')
#print("Visible GPUs:", gpus)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs", len(logical_gpus), "logical GPUs")
  except RuntimeError as e:
    print(e)

print("Tf will attempt to allocate only as much GPU Memory as needed for runtime allowed")


In [ ]:
def convert(seconds): 
    return time.strftime("%H:%M:%S", time.gmtime(seconds)) 


def get_model_memory_usage(batch_size, model):
    import numpy as np
    try:
        from keras import backend as K
    except:
        from tensorflow.keras import backend as K

    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(p) for p in model.non_trainable_weights])

    number_size = 4.0
    if K.floatx() == 'float16':
        number_size = 2.0
    if K.floatx() == 'float64':
        number_size = 8.0

    total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes


In [ ]:
train_dir = r'/home/viraat/Downloads/Rejoy/ekush_train_val_test_split/train'
test_dir= r'/home/viraat/Downloads/Rejoy/ekush_train_val_test_split/test'
val_dir=r'/home/viraat/Downloads/Rejoy/ekush_train_val_test_split/val'


save_file_file= r'ekush_multiscale.keras'

In [ ]:
img_h,img_w= (32,32)
batch_size=20
epochs=100
nb_classes=122


train_datagen= tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen= tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(img_h, img_w),  
        batch_size=batch_size,
        class_mode='categorical',
        color_mode="rgb",
        shuffle=True)


test_generator = test_datagen.flow_from_directory(
         test_dir,     
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode="rgb",
        shuffle=False)

val_generator = val_datagen.flow_from_directory(
         val_dir,     
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode="rgb",
        shuffle=False)

In [ ]:
input_shape = (img_h, img_w, 3)

input_img = tf.keras.layers.Input(shape=input_shape)
conv1 = tf.keras.layers.Conv2D(64, kernel_size=(5, 5),  padding='same', activation='relu')(input_img)
conv2 = tf.keras.layers.Conv2D(64, kernel_size=(5, 5),  padding='same', activation='relu')(conv1)
conv3 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  padding='same', activation='relu')(conv1)
conv4 = tf.keras.layers.Conv2D(64, kernel_size=(1, 1),  padding='same', activation='relu')(conv1)


conv5 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  padding='same', activation='relu')(conv2)
conv6 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  padding='same', activation='relu')(conv3)
conv7 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  padding='same', activation='relu')(conv4)

conv8 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv5)
conv9 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv6)
conv10 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv7)

added1 = tf.keras.layers.Add()([conv8,conv9,conv10])
added1 = tf.keras.layers.BatchNormalization()(added1)

conv11 = tf.keras.layers.Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu')(added1)
conv12 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  padding='same', activation='relu')(added1)
conv13 = tf.keras.layers.Conv2D(64, kernel_size=(1, 1),  padding='same', activation='relu')(added1)

conv14 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv11)
conv15 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv12)
conv16 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv13)

conv17 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv14)
conv18 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv15)
conv19 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv16)

added2 = tf.keras.layers.Add()([conv17,conv18,conv19])
added2 = tf.keras.layers.BatchNormalization()(added2)

conv20 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(added2)
conv21 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(added2)
conv22 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(added2)

conv23 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv20)
conv24 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv21)
conv25 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(conv22)

concatenated = tf.keras.layers.Concatenate()([conv23,conv24,conv25])
concatenated = tf.keras.layers.BatchNormalization()(concatenated)

conv26 = tf.keras.layers.Conv2D(128, kernel_size=(5, 5), strides=(2,2),padding='same', activation='relu')(concatenated)
conv27 = tf.keras.layers.Conv2D(128, kernel_size=(3, 3),  strides=(2,2),padding='same', activation='relu')(concatenated)
conv28 = tf.keras.layers.Conv2D(128, kernel_size=(1, 1), strides=(2,2), padding='same', activation='relu')(concatenated)

added3 = tf.keras.layers.Add()([conv26,conv27,conv28])

conv29 = tf.keras.layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(added3)
conv30 = tf.keras.layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(conv29) ##If works, then remove this and recheck
conv30 = tf.keras.layers.BatchNormalization()(conv30)

flatten = tf.keras.layers.Flatten()(conv30)


fully_connected_1 = tf.keras.layers.Dense(1024, activation='relu')(flatten)
fully_connected_2 = tf.keras.layers.Dense(512, activation='relu')(fully_connected_1)
fully_connected_2 = tf.keras.layers.Dropout(0.5)(fully_connected_2)
fully_connected_3 = tf.keras.layers.Dense(256, activation='relu')(fully_connected_2)
output = tf.keras.layers.Dense(122, activation='softmax')(fully_connected_3)


model = tf.keras.models.Model(inputs = input_img, outputs=output,name="ekush_multiscale")

In [ ]:
model.summary()

In [ ]:
reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.1,
                                         patience=5,
                                         cooldown=2,
                                         min_lr=1e-10,
                                         verbose=1)

checkpoint =tf.keras.callbacks.ModelCheckpoint(filepath=save_file_file, 
                            monitor='val_accuracy',
                            verbose=1,
                            save_best_only=True, 
                            save_weights_only=False, 
                            mode='auto',
                            save_freq='epoch')

early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=25, verbose=1, mode='auto')

In [ ]:

#TIMING CALL BACK STORING TIME OF EACH EPOCH
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={'log.txt'}):
        self.times = []

    def on_epoch_begin(self, batch, logs={'log.txt'}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={'log.txt'}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

In [ ]:
callbacks = [reduce_learning_rate, checkpoint, time_callback, early]
optimizer =tf.keras.optimizers.Adam(learning_rate=0.0001,epsilon=1e-08)
model.compile( loss='categorical_crossentropy',optimizer= optimizer, metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
start = time.time()
history = model.fit(
      train_generator, 
      epochs=epochs,
      validation_data= val_generator,
      callbacks=callbacks,
      verbose=1)
end = time.time()

In [ ]:
execution_time_ms = (end - start) * 10**3

hours = execution_time_ms // (1000 * 60 * 60)
execution_time_ms %= (1000 * 60 * 60)

minutes = execution_time_ms // (1000 * 60)
execution_time_ms %= (1000 * 60)

seconds = execution_time_ms // 1000

print("The time of execution of above program is: {} hours, {} minutes, and {} seconds.".format(int(hours), int(minutes), int(seconds)))

In [ ]:
prediction = model.predict(test_generator,verbose=1)
pred_vals= np.argmax(prediction,axis=1)
true_lebels=test_generator.classes

In [ ]:
print("The accuracy is: " + str(accuracy_score(true_lebels,pred_vals)*100))

In [ ]:
cf=confusion_matrix(true_lebels, pred_vals)
print(cf)

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.plot(range(1, len(train_acc) + 1), train_acc, label="Training")
plt.plot(range(1, len(val_acc) + 1), val_acc, label="Validation")  # Added comma and label
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.title("Accuracy Plot")
plt.legend()
plt.savefig("plot_acc.pdf")


In [ ]:
plt.plot(range(1, len(train_loss) + 1), train_loss, label="Training")
plt.plot(range(1, len(val_loss) + 1), val_loss, label="Validation")  # Added comma and label
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.title("Loss Plot")
plt.legend()
plt.savefig("plot_loss.pdf")

In [ ]:
train_prec = history.history['precision']
val_prec = history.history['val_precision']
train_recall = history.history['recall']
val_recall = history.history['val_recall']

In [ ]:
plt.plot(range(1, len(train_prec) + 1), train_prec, label="Training")
plt.plot(range(1, len(val_prec) + 1), val_prec, label="Validation")  
plt.ylabel("Precision")
plt.xlabel("Epoch")
plt.title("Precision Plot")
plt.legend()
plt.savefig("plot_prec.pdf")

In [ ]:
plt.plot(range(1, len(train_recall) + 1), train_recall, label="Training")
plt.plot(range(1, len(val_recall) + 1), val_recall, label="Validation")
plt.ylabel("Recall")
plt.xlabel("Epoch")
plt.title("Recall Plot")
plt.legend()
plt.savefig("plot_recall.pdf")

In [ ]:
import seaborn as sns

sns.heatmap(cf, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=False, yticklabels=False)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (Without Class Labels)')
plt.show()